In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import re
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, plot_roc_curve, balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
import xgboost
import lightgbm
from sklearn import linear_model, tree, ensemble
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings(action='ignore')
pd.options.display.max_columns = None
import math

In [2]:
train=pd.read_csv('phase2_train.csv', index_col=0)
val=pd.read_csv('phase2_test.csv', index_col=0)

In [3]:
train.loc[train['User_ID'] <= 240, 'Gender'] = train.loc[train['User_ID'] <= 240, 'Gender'].replace({0: 1, 1: 0})

In [4]:
train

,User_ID,Gender,Age,MBTI,Q_number,Answer
0,1,0,30,INFP,1,아니다. 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의 ...
1,1,0,30,INFP,2,중립. 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하며...
2,1,0,30,INFP,3,그렇다. 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 영...
3,1,0,30,INFP,4,중립. 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고 ...
4,1,0,30,INFP,5,아니다. 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대로...
...,...,...,...,...,...,...
18715,480,0,30,ISTP,56,그렇다. 거래처에 가격 조정 때문에 3군데를 가야 하는 상황이었는데 이야기 잘 통하...
18716,480,0,30,ISTP,57,아니다. 상대방과 논쟁을 불러드릴 주제에는 관심이 없습니다 괜히 싸움을 일으키기 싫습니다
18717,480,0,30,ISTP,58,아니다. 나에게 온 기회를 포기할 수 없다 양보를 하게 되면 나에게 기회는 없어지니깐
18718,480,0,30,ISTP,59,아니다. 마감 기한이 정해지면 그 일을 끝날 때까지 늦게까지 일을 하고 퇴근하곤 합니다


In [5]:
val

,User_ID,Gender,Age,Q_number,Answer
0,241,0,30,1,아니다. 친구를 만들 상황에 새로운 친구를 만듭니다. 의도적으로나 꼭 주기적으로 새...
1,241,0,30,2,그렇다. 일상이 무료한 걸 못 참는 편. 요즘은 리본으로 뜨개질해서 가방 만드는 걸...
2,241,0,30,3,아니다. 다른 사람이 울고 있는 모습을 본다고 해도 눈물이 나지는 않습니다. 그 심...
3,241,0,30,4,그렇다. 진행하고 있는 일이 계획단계일 때 이미 대비책을 함께 세우는 편. 잘못된 ...
4,241,0,30,5,"그렇다. 최근 프로젝트 진행 중 문제가 발생, 마감기한이 얼마 남지 않았고, 주변 ..."
...,...,...,...,...,...
7195,453,0,20,56,아니다. 저는 절차대로 일을 완수하는 편이 아닙니다. 이유는 절차적으로 일을 완수하...
7196,453,0,20,57,아니다. 저는 논쟁을 불러일으킬 주제에 관심이 없습니다. 이유는 논쟁을 불러일으킬 ...
7197,453,0,20,58,아니다. 저는 다른 사람에게 더 필요한 기회라고 생각되면 기회를 포기할 수 없습니다...
7198,453,0,20,59,아니다. 저는 마감 기한을 지키기 쉽습니다. 이유는 주어진 기한 안에 문제를 해결하...


In [6]:
def col(df):
    mapping = {'I': 0, 'E': 1, 'S': 0, 'N': 1, 'T': 0, 'F': 1, 'J': 0, 'P': 1}

    df['I/E'] = df['MBTI'].apply(lambda x: mapping[x[0]])
    df['S/N'] = df['MBTI'].apply(lambda x: mapping[x[1]])
    df['T/F'] = df['MBTI'].apply(lambda x: mapping[x[2]])
    df['J/P'] = df['MBTI'].apply(lambda x: mapping[x[3]])
    return df

train=col(train)
train

,User_ID,Gender,Age,MBTI,Q_number,Answer,I/E,S/N,T/F,J/P
0,1,0,30,INFP,1,아니다. 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의 ...,0,1,1,1
1,1,0,30,INFP,2,중립. 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하며...,0,1,1,1
2,1,0,30,INFP,3,그렇다. 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 영...,0,1,1,1
3,1,0,30,INFP,4,중립. 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고 ...,0,1,1,1
4,1,0,30,INFP,5,아니다. 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대로...,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
18715,480,0,30,ISTP,56,그렇다. 거래처에 가격 조정 때문에 3군데를 가야 하는 상황이었는데 이야기 잘 통하...,0,0,0,1
18716,480,0,30,ISTP,57,아니다. 상대방과 논쟁을 불러드릴 주제에는 관심이 없습니다 괜히 싸움을 일으키기 싫습니다,0,0,0,1
18717,480,0,30,ISTP,58,아니다. 나에게 온 기회를 포기할 수 없다 양보를 하게 되면 나에게 기회는 없어지니깐,0,0,0,1
18718,480,0,30,ISTP,59,아니다. 마감 기한이 정해지면 그 일을 끝날 때까지 늦게까지 일을 하고 퇴근하곤 합니다,0,0,0,1


In [7]:
#train=train[train['Q_number']<49]

#val=val[val['Q_number']<49]

In [8]:
train['Answer'] = train['Answer'].str.split('.').str[0]
val['Answer'] = val['Answer'].str.split('.').str[0]

In [9]:
train

,User_ID,Gender,Age,MBTI,Q_number,Answer,I/E,S/N,T/F,J/P
0,1,0,30,INFP,1,아니다,0,1,1,1
1,1,0,30,INFP,2,중립,0,1,1,1
2,1,0,30,INFP,3,그렇다,0,1,1,1
3,1,0,30,INFP,4,중립,0,1,1,1
4,1,0,30,INFP,5,아니다,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
18715,480,0,30,ISTP,56,그렇다,0,0,0,1
18716,480,0,30,ISTP,57,아니다,0,0,0,1
18717,480,0,30,ISTP,58,아니다,0,0,0,1
18718,480,0,30,ISTP,59,아니다,0,0,0,1


In [10]:
def merge(df):
    df['Q_number'] = 'Question' + df['Q_number'].astype(str)

    pivot_df = df.pivot(index=['User_ID', 'Gender', 'Age', 'I/E', 'S/N', 'T/F', 'J/P'], columns='Q_number', values='Answer')

    pivot_df = pivot_df.reset_index()
    return pivot_df

def merge2(df):
    df['Q_number'] = 'Question' + df['Q_number'].astype(str)

    pivot_df = df.pivot(index=['User_ID', 'Gender', 'MBTI', 'Age', 'I/E', 'S/N', 'T/F', 'J/P'], columns='Q_number', values='Answer')

    pivot_df = pivot_df.reset_index()
    return pivot_df

In [11]:
val['I/E']=np.nan
val['S/N']=np.nan
val['T/F']=np.nan
val['J/P']=np.nan
val

,User_ID,Gender,Age,Q_number,Answer,I/E,S/N,T/F,J/P
0,241,0,30,1,아니다,NaN,NaN,NaN,NaN
1,241,0,30,2,그렇다,NaN,NaN,NaN,NaN
2,241,0,30,3,아니다,NaN,NaN,NaN,NaN
3,241,0,30,4,그렇다,NaN,NaN,NaN,NaN
4,241,0,30,5,그렇다,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7195,453,0,20,56,아니다,NaN,NaN,NaN,NaN
7196,453,0,20,57,아니다,NaN,NaN,NaN,NaN
7197,453,0,20,58,아니다,NaN,NaN,NaN,NaN
7198,453,0,20,59,아니다,NaN,NaN,NaN,NaN


In [12]:
train['Answer'] = train['Answer'].str.replace(' ', '')
train['Answer'] = train['Answer'].str.replace(',', '')

val['Answer'] = val['Answer'].str.replace(' ', '')
val['Answer'] = val['Answer'].str.replace(',', '')

train['Answer'] = train['Answer'].str[0]
val['Answer'] = val['Answer'].str[0]

train['Answer'] = train['Answer'].replace('어', '아')

In [13]:
train=merge(train)
val=merge(val)

In [14]:
train

Q_number,User_ID,Gender,Age,I/E,S/N,T/F,J/P,Question1,Question10,Question11,Question12,Question13,Question14,Question15,Question16,Question17,Question18,Question19,Question2,Question20,Question21,Question22,Question23,Question24,Question25,Question26,Question27,Question28,Question29,Question3,Question30,Question31,Question32,Question33,Question34,Question35,Question36,Question37,Question38,Question39,Question4,Question40,Question41,Question42,Question43,Question44,Question45,Question46,Question47,Question48,Question49,Question5,Question50,Question51,Question52,Question53,Question54,Question55,Question56,Question57,Question58,Question59,Question6,Question60,Question7,Question8,Question9
0,1,0,30,0,1,1,1,아,중,아,아,중,중,중,아,아,중,그,중,아,중,아,중,아,그,그,그,그,그,그,그,아,그,그,그,아,아,그,아,아,중,아,그,그,아,그,그,그,그,중,NaN,아,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,그,NaN,아,그,중
1,2,0,40,1,0,0,0,중,중,중,아,그,아,중,중,중,그,그,그,중,아,아,아,그,아,아,그,그,아,그,아,중,중,그,아,중,그,그,중,그,그,아,그,아,중,아,그,그,그,중,NaN,아,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,아,NaN,그,중,그
2,3,1,40,0,0,0,1,아,그,아,그,그,그,중,아,아,아,아,아,그,그,그,그,그,중,그,아,그,중,아,아,중,그,아,그,중,아,아,그,그,그,중,중,아,아,중,그,그,중,중,NaN,중,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,그,NaN,중,아,중
3,4,0,40,0,1,1,0,아,그,중,그,그,아,중,중,그,중,아,그,그,그,아,그,중,그,아,그,그,그,그,중,아,중,그,그,아,아,그,아,그,그,그,아,중,아,그,그,아,그,중,NaN,그,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,그,NaN,그,그,그
4,5,0,40,0,0,0,0,아,아,아,그,그,아,아,아,아,아,아,그,그,그,아,중,그,중,그,아,중,그,아,아,아,그,중,그,아,아,아,그,그,그,중,그,아,아,아,그,그,아,그,NaN,그,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,그,NaN,그,아,아
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,476,1,30,1,0,1,1,그,중,그,아,중,그,그,아,그,아,그,그,아,아,아,아,아,아,중,그,중,그,중,그,그,중,그,아,그,그,그,아,아,아,그,아,아,그,중,그,아,중,그,그,그,아,아,그,그,그,아,아,그,아,그,중,그,아,그,아
356,477,1,40,1,0,1,0,중,아,그,중,그,그,그,중,그,중,아,그,아,아,그,그,아,그,아,아,그,그,아,그,중,아,아,그,그,아,아,그,아,아,그,중,그,아,아,아,중,아,중,그,아,아,그,아,중,아,아,중,그,중,그,아,그,그,아,중
357,478,0,30,1,0,0,0,그,그,그,아,아,그,아,그,그,그,아,그,그,아,아,중,그,아,그,아,그,그,그,그,아,그,그,그,아,아,그,아,아,아,아,아,아,그,아,그,아,중,그,그,그,그,그,아,아,그,아,아,그,그,그,아,아,아,그,아
358,479,1,40,0,1,1,1,그,아,그,중,아,그,그,그,그,중,아,그,아,중,아,아,아,그,중,그,중,그,아,그,그,그,그,아,그,그,중,아,중,아,그,아,아,그,아,아,그,중,아,그,그,아,아,중,그,그,아,아,그,중,아,아,그,중,아,아


In [15]:
train=train[train['User_ID']>240]

In [16]:
val

Q_number,User_ID,Gender,Age,I/E,S/N,T/F,J/P,Question1,Question10,Question11,Question12,Question13,Question14,Question15,Question16,Question17,Question18,Question19,Question2,Question20,Question21,Question22,Question23,Question24,Question25,Question26,Question27,Question28,Question29,Question3,Question30,Question31,Question32,Question33,Question34,Question35,Question36,Question37,Question38,Question39,Question4,Question40,Question41,Question42,Question43,Question44,Question45,Question46,Question47,Question48,Question49,Question5,Question50,Question51,Question52,Question53,Question54,Question55,Question56,Question57,Question58,Question59,Question6,Question60,Question7,Question8,Question9
0,241,0,30,NaN,NaN,NaN,NaN,아,그,그,아,그,아,그,그,아,아,그,그,아,중,아,그,그,아,그,아,그,중,아,아,그,그,중,아,그,그,그,중,중,그,아,아,중,그,그,그,아,아,중,아,그,그,아,아,아,중,아,아,아,아,아,중,아,아,아,그
1,242,1,30,NaN,NaN,NaN,NaN,아,아,아,아,그,그,그,그,아,그,아,그,아,그,아,그,아,그,아,그,아,그,중,그,중,아,그,아,그,아,아,아,아,그,그,아,아,그,아,아,아,아,그,그,그,아,아,아,중,그,아,그,그,그,아,그,그,아,중,아
2,243,0,40,NaN,NaN,NaN,NaN,중,아,중,그,그,아,아,그,아,아,아,아,아,그,그,아,아,아,중,아,아,아,아,그,그,그,아,중,그,아,아,아,그,그,그,아,아,아,아,아,그,아,아,아,그,그,아,그,그,아,아,그,아,아,아,그,그,그,아,그
3,244,0,30,NaN,NaN,NaN,NaN,아,아,그,그,그,중,그,중,아,아,아,그,아,그,아,아,그,아,그,아,중,아,그,아,그,그,중,아,그,중,그,아,그,그,그,아,중,중,중,아,아,아,중,아,중,아,아,아,그,그,아,중,아,그,아,중,그,아,아,그
4,245,0,30,NaN,NaN,NaN,NaN,중,아,그,그,중,중,그,아,아,아,아,중,아,아,그,중,아,아,아,그,그,그,그,그,중,아,아,그,그,그,중,아,그,그,그,아,중,그,그,그,아,그,중,아,그,그,아,아,아,그,중,아,아,아,아,중,그,아,그,그
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,415,1,40,NaN,NaN,NaN,NaN,아,그,그,중,아,아,아,아,아,아,아,아,그,그,아,중,그,중,그,그,그,아,그,아,중,그,아,중,그,아,그,아,그,아,아,그,아,중,아,아,그,그,아,아,아,그,그,그,아,그,중,중,중,그,아,그,중,그,그,중
116,418,1,40,NaN,NaN,NaN,NaN,중,아,그,아,아,중,그,중,그,아,아,그,아,중,아,그,아,중,아,중,아,아,아,그,중,아,중,아,그,아,그,중,중,그,그,그,그,아,아,중,아,아,아,아,그,아,그,아,아,그,아,그,중,아,아,중,그,아,그,그
117,419,1,20,NaN,NaN,NaN,NaN,아,중,중,그,아,중,중,그,중,중,아,아,그,그,그,중,아,아,그,아,중,중,그,아,그,그,중,중,중,중,아,중,아,중,중,중,아,중,아,그,중,그,그,그,아,중,중,아,아,중,중,아,아,그,아,그,중,그,중,중
118,431,0,30,NaN,NaN,NaN,NaN,아,아,아,그,그,아,아,그,아,아,그,그,그,아,그,그,그,아,그,중,그,그,중,아,아,그,아,그,아,아,아,그,그,그,그,아,아,그,그,그,그,아,그,아,그,아,그,그,아,그,아,아,아,그,아,그,그,아,중,그


In [17]:
train=train.drop(['User_ID'], axis=1)
val=val.drop('User_ID', axis=1)
train

Q_number,Gender,Age,I/E,S/N,T/F,J/P,Question1,Question10,Question11,Question12,Question13,Question14,Question15,Question16,Question17,Question18,Question19,Question2,Question20,Question21,Question22,Question23,Question24,Question25,Question26,Question27,Question28,Question29,Question3,Question30,Question31,Question32,Question33,Question34,Question35,Question36,Question37,Question38,Question39,Question4,Question40,Question41,Question42,Question43,Question44,Question45,Question46,Question47,Question48,Question49,Question5,Question50,Question51,Question52,Question53,Question54,Question55,Question56,Question57,Question58,Question59,Question6,Question60,Question7,Question8,Question9
240,0,30,0,1,0,0,아,아,그,아,그,아,중,아,중,중,아,그,아,중,아,그,그,중,중,아,그,아,그,아,아,그,아,아,그,아,그,아,그,그,중,중,아,아,중,중,중,아,중,아,그,중,그,중,아,그,아,그,중,중,아,그,그,중,아,그
241,0,40,1,1,0,0,중,그,그,그,그,중,그,아,아,아,아,그,그,그,그,중,그,아,그,중,그,아,그,아,아,중,중,그,아,아,아,아,그,그,아,아,아,아,아,그,아,그,아,아,아,아,그,중,아,아,아,그,중,아,아,중,아,중,그,그
242,0,30,1,0,1,0,아,아,중,그,그,그,아,그,중,아,아,그,그,중,그,그,그,그,아,그,그,아,그,아,중,그,중,그,아,중,아,아,중,중,아,아,아,그,그,그,그,그,그,그,그,중,그,그,중,그,아,그,그,아,아,그,그,그,중,아
243,0,40,1,1,1,0,아,그,중,아,중,그,아,아,중,그,그,그,아,그,아,중,중,그,그,중,그,그,그,그,아,중,그,그,그,아,그,아,그,그,아,아,그,중,아,아,아,아,아,그,그,그,그,아,아,그,그,중,그,그,아,그,그,중,그,중
244,0,40,1,0,1,0,그,중,그,그,중,중,중,중,그,그,그,그,아,아,아,그,중,중,중,그,그,아,중,그,그,그,중,중,그,그,그,중,중,아,그,아,그,중,그,아,아,중,그,그,그,그,중,아,중,그,그,아,아,중,아,중,그,아,그,중
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,1,30,1,0,1,1,그,중,그,아,중,그,그,아,그,아,그,그,아,아,아,아,아,아,중,그,중,그,중,그,그,중,그,아,그,그,그,아,아,아,그,아,아,그,중,그,아,중,그,그,그,아,아,그,그,그,아,아,그,아,그,중,그,아,그,아
356,1,40,1,0,1,0,중,아,그,중,그,그,그,중,그,중,아,그,아,아,그,그,아,그,아,아,그,그,아,그,중,아,아,그,그,아,아,그,아,아,그,중,그,아,아,아,중,아,중,그,아,아,그,아,중,아,아,중,그,중,그,아,그,그,아,중
357,0,30,1,0,0,0,그,그,그,아,아,그,아,그,그,그,아,그,그,아,아,중,그,아,그,아,그,그,그,그,아,그,그,그,아,아,그,아,아,아,아,아,아,그,아,그,아,중,그,그,그,그,그,아,아,그,아,아,그,그,그,아,아,아,그,아
358,1,40,0,1,1,1,그,아,그,중,아,그,그,그,그,중,아,그,아,중,아,아,아,그,중,그,중,그,아,그,그,그,그,아,그,그,중,아,중,아,그,아,아,그,아,아,그,중,아,그,그,아,아,중,그,그,아,아,그,중,아,아,그,중,아,아


In [18]:
train['Question1'].value_counts()

아    61
그    35
중    24
Name: Question1, dtype: int64

In [19]:
train = train.replace('아', -1)
train = train.replace('그', 1)
train = train.replace('중', 0)

val = val.replace('아', -1)
val = val.replace('그', 1)
val = val.replace('중', 0)

In [20]:
train=train.reset_index(drop=True)

In [21]:
train

Q_number,Gender,Age,I/E,S/N,T/F,J/P,Question1,Question10,Question11,Question12,Question13,Question14,Question15,Question16,Question17,Question18,Question19,Question2,Question20,Question21,Question22,Question23,Question24,Question25,Question26,Question27,Question28,Question29,Question3,Question30,Question31,Question32,Question33,Question34,Question35,Question36,Question37,Question38,Question39,Question4,Question40,Question41,Question42,Question43,Question44,Question45,Question46,Question47,Question48,Question49,Question5,Question50,Question51,Question52,Question53,Question54,Question55,Question56,Question57,Question58,Question59,Question6,Question60,Question7,Question8,Question9
0,0,30,0,1,0,0,-1,-1,1,-1,1,-1,0,-1,0,0,-1,1,-1,0,-1,1.0,1,0,0,-1,1,-1,1,-1,-1,1,-1,-1,1,-1,1,-1,1,1,0,0,-1,-1,0.0,0,0,-1,0,-1,1,0,1,0,-1,1,-1,1,0,0,-1,1,1,0,-1,1
1,0,40,1,1,0,0,0,1,1,1,1,0,1,-1,-1,-1,-1,1,1,1,1,0.0,1,-1,1,0,1,-1,1,-1,-1,0,0,1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1.0,1,-1,1,-1,-1,-1,-1,1,0,-1,-1,-1,1,0,-1,-1,0,-1,0,1,1
2,0,30,1,0,1,0,-1,-1,0,1,1,1,-1,1,0,-1,-1,1,1,0,1,1.0,1,1,-1,1,1,-1,1,-1,0,1,0,1,-1,0,-1,-1,0,0,-1,-1,-1,1,1.0,1,1,1,1,1,1,0,1,1,0,1,-1,1,1,-1,-1,1,1,1,0,-1
3,0,40,1,1,1,0,-1,1,0,-1,0,1,-1,-1,0,1,1,1,-1,1,-1,0.0,0,1,1,0,1,1,1,1,-1,0,1,1,1,-1,1,-1,1,1,-1,-1,1,0,-1.0,-1,-1,-1,-1,1,1,1,1,-1,-1,1,1,0,1,1,-1,1,1,0,1,0
4,0,40,1,0,1,0,1,0,1,1,0,0,0,0,1,1,1,1,-1,-1,-1,1.0,0,0,0,1,1,-1,0,1,1,1,0,0,1,1,1,0,0,-1,1,-1,1,0,1.0,-1,-1,0,1,1,1,1,0,-1,0,1,1,-1,-1,0,-1,0,1,-1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,30,1,0,1,1,1,0,1,-1,0,1,1,-1,1,-1,1,1,-1,-1,-1,-1.0,-1,-1,0,1,0,1,0,1,1,0,1,-1,1,1,1,-1,-1,-1,1,-1,-1,1,0.0,1,-1,0,1,1,1,-1,-1,1,1,1,-1,-1,1,-1,1,0,1,-1,1,-1
116,1,40,1,0,1,0,0,-1,1,0,1,1,1,0,1,0,-1,1,-1,-1,1,1.0,-1,1,-1,-1,1,1,-1,1,0,-1,-1,1,1,-1,-1,1,-1,-1,1,0,1,-1,-1.0,-1,0,-1,0,1,-1,-1,1,-1,0,-1,-1,0,1,0,1,-1,1,1,-1,0
117,0,30,1,0,0,0,1,1,1,-1,-1,1,-1,1,1,1,-1,1,1,-1,-1,0.0,1,-1,1,-1,1,1,1,1,-1,1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1.0,1,-1,0,1,1,1,1,1,-1,-1,1,-1,-1,1,1,1,-1,-1,-1,1,-1
118,1,40,0,1,1,1,1,-1,1,0,-1,1,1,1,1,0,-1,1,-1,0,-1,-1.0,-1,1,0,1,0,1,-1,1,1,1,1,-1,1,1,0,-1,0,-1,1,-1,-1,1,-1.0,-1,1,0,-1,1,1,-1,-1,0,1,1,-1,-1,1,0,-1,-1,1,0,-1,-1


In [22]:
#train = train.dropna(axis=1)

In [23]:
train

Q_number,Gender,Age,I/E,S/N,T/F,J/P,Question1,Question10,Question11,Question12,Question13,Question14,Question15,Question16,Question17,Question18,Question19,Question2,Question20,Question21,Question22,Question23,Question24,Question25,Question26,Question27,Question28,Question29,Question3,Question30,Question31,Question32,Question33,Question34,Question35,Question36,Question37,Question38,Question39,Question4,Question40,Question41,Question42,Question43,Question44,Question45,Question46,Question47,Question48,Question49,Question5,Question50,Question51,Question52,Question53,Question54,Question55,Question56,Question57,Question58,Question59,Question6,Question60,Question7,Question8,Question9
0,0,30,0,1,0,0,-1,-1,1,-1,1,-1,0,-1,0,0,-1,1,-1,0,-1,1.0,1,0,0,-1,1,-1,1,-1,-1,1,-1,-1,1,-1,1,-1,1,1,0,0,-1,-1,0.0,0,0,-1,0,-1,1,0,1,0,-1,1,-1,1,0,0,-1,1,1,0,-1,1
1,0,40,1,1,0,0,0,1,1,1,1,0,1,-1,-1,-1,-1,1,1,1,1,0.0,1,-1,1,0,1,-1,1,-1,-1,0,0,1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1.0,1,-1,1,-1,-1,-1,-1,1,0,-1,-1,-1,1,0,-1,-1,0,-1,0,1,1
2,0,30,1,0,1,0,-1,-1,0,1,1,1,-1,1,0,-1,-1,1,1,0,1,1.0,1,1,-1,1,1,-1,1,-1,0,1,0,1,-1,0,-1,-1,0,0,-1,-1,-1,1,1.0,1,1,1,1,1,1,0,1,1,0,1,-1,1,1,-1,-1,1,1,1,0,-1
3,0,40,1,1,1,0,-1,1,0,-1,0,1,-1,-1,0,1,1,1,-1,1,-1,0.0,0,1,1,0,1,1,1,1,-1,0,1,1,1,-1,1,-1,1,1,-1,-1,1,0,-1.0,-1,-1,-1,-1,1,1,1,1,-1,-1,1,1,0,1,1,-1,1,1,0,1,0
4,0,40,1,0,1,0,1,0,1,1,0,0,0,0,1,1,1,1,-1,-1,-1,1.0,0,0,0,1,1,-1,0,1,1,1,0,0,1,1,1,0,0,-1,1,-1,1,0,1.0,-1,-1,0,1,1,1,1,0,-1,0,1,1,-1,-1,0,-1,0,1,-1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,30,1,0,1,1,1,0,1,-1,0,1,1,-1,1,-1,1,1,-1,-1,-1,-1.0,-1,-1,0,1,0,1,0,1,1,0,1,-1,1,1,1,-1,-1,-1,1,-1,-1,1,0.0,1,-1,0,1,1,1,-1,-1,1,1,1,-1,-1,1,-1,1,0,1,-1,1,-1
116,1,40,1,0,1,0,0,-1,1,0,1,1,1,0,1,0,-1,1,-1,-1,1,1.0,-1,1,-1,-1,1,1,-1,1,0,-1,-1,1,1,-1,-1,1,-1,-1,1,0,1,-1,-1.0,-1,0,-1,0,1,-1,-1,1,-1,0,-1,-1,0,1,0,1,-1,1,1,-1,0
117,0,30,1,0,0,0,1,1,1,-1,-1,1,-1,1,1,1,-1,1,1,-1,-1,0.0,1,-1,1,-1,1,1,1,1,-1,1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1.0,1,-1,0,1,1,1,1,1,-1,-1,1,-1,-1,1,1,1,-1,-1,-1,1,-1
118,1,40,0,1,1,1,1,-1,1,0,-1,1,1,1,1,0,-1,1,-1,0,-1,-1.0,-1,1,0,1,0,1,-1,1,1,1,1,-1,1,1,0,-1,0,-1,1,-1,-1,1,-1.0,-1,1,0,-1,1,1,-1,-1,0,1,1,-1,-1,1,0,-1,-1,1,0,-1,-1


In [24]:
#train=pd.concat([train, train, train, train], ignore_index=True)

In [25]:
def dummy_creation(df, columns_list):
    df_dummies = []
    for col in columns_list:
        df_dummies.append(pd.get_dummies(df[col], prefix = col, prefix_sep = ':'))
    df_dummies = pd.concat(df_dummies, axis = 1)
    df = pd.concat([df, df_dummies], axis = 1)
    return df

In [26]:
#train=train[train['Q_number']==1]
#train

In [27]:
def model_perf(model, X, Y, prefix=''):
    p_hat = model.predict_proba(X)[:, 1]
    Y = Y.astype(int)
    auc = roc_auc_score(Y, p_hat)
    print (f'{prefix} AUROC score = {auc:.3f}')
    return auc

# note : cross validation is not included because of computation time
def model_fit_perf(model, X_all, Y_all, prefix=''):
    X_trn, X_val, Y_trn, Y_val = train_test_split(X_all, Y_all, test_size=0.2, random_state=4)
    model.fit(X_trn, Y_trn)
    print (prefix)
    model_perf(model, X_trn, Y_trn, '  Training  ')
    model_perf(model, X_val, Y_val, '  Validation')

In [28]:
#train = dummy_creation(train, ['Gender', 'Age'])
#val = dummy_creation(val, ['Gender', 'Age'])


Y_train1=train['I/E']
Y_train2=train['S/N']
Y_train3=train['T/F']
Y_train4=train['J/P']

Y_val1=val['I/E']
Y_val2=val['S/N']
Y_val3=val['T/F']
Y_val4=val['J/P']

train=train.drop(columns=['Gender', 'Age', 'I/E', 'S/N', 'T/F', 'J/P'])
val=val.drop(columns=['Gender', 'Age', 'I/E', 'S/N', 'T/F', 'J/P'])

In [29]:
train=train.fillna(0)

In [43]:
train

Q_number,Question1,Question10,Question11,Question12,Question13,Question14,Question15,Question16,Question17,Question18,Question19,Question2,Question20,Question21,Question22,Question23,Question24,Question25,Question26,Question27,Question28,Question29,Question3,Question30,Question31,Question32,Question33,Question34,Question35,Question36,Question37,Question38,Question39,Question4,Question40,Question41,Question42,Question43,Question44,Question45,Question46,Question47,Question48,Question49,Question5,Question50,Question51,Question52,Question53,Question54,Question55,Question56,Question57,Question58,Question59,Question6,Question60,Question7,Question8,Question9
0,-1,-1,1,-1,1,-1,0,-1,0,0,-1,1,-1,0,-1,1.0,1,0,0,-1,1,-1,1,-1,-1,1,-1,-1,1,-1,1,-1,1,1,0,0,-1,-1,0.0,0,0,-1,0,-1,1,0,1,0,-1,1,-1,1,0,0,-1,1,1,0,-1,1
1,0,1,1,1,1,0,1,-1,-1,-1,-1,1,1,1,1,0.0,1,-1,1,0,1,-1,1,-1,-1,0,0,1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1.0,1,-1,1,-1,-1,-1,-1,1,0,-1,-1,-1,1,0,-1,-1,0,-1,0,1,1
2,-1,-1,0,1,1,1,-1,1,0,-1,-1,1,1,0,1,1.0,1,1,-1,1,1,-1,1,-1,0,1,0,1,-1,0,-1,-1,0,0,-1,-1,-1,1,1.0,1,1,1,1,1,1,0,1,1,0,1,-1,1,1,-1,-1,1,1,1,0,-1
3,-1,1,0,-1,0,1,-1,-1,0,1,1,1,-1,1,-1,0.0,0,1,1,0,1,1,1,1,-1,0,1,1,1,-1,1,-1,1,1,-1,-1,1,0,-1.0,-1,-1,-1,-1,1,1,1,1,-1,-1,1,1,0,1,1,-1,1,1,0,1,0
4,1,0,1,1,0,0,0,0,1,1,1,1,-1,-1,-1,1.0,0,0,0,1,1,-1,0,1,1,1,0,0,1,1,1,0,0,-1,1,-1,1,0,1.0,-1,-1,0,1,1,1,1,0,-1,0,1,1,-1,-1,0,-1,0,1,-1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,0,1,-1,0,1,1,-1,1,-1,1,1,-1,-1,-1,-1.0,-1,-1,0,1,0,1,0,1,1,0,1,-1,1,1,1,-1,-1,-1,1,-1,-1,1,0.0,1,-1,0,1,1,1,-1,-1,1,1,1,-1,-1,1,-1,1,0,1,-1,1,-1
116,0,-1,1,0,1,1,1,0,1,0,-1,1,-1,-1,1,1.0,-1,1,-1,-1,1,1,-1,1,0,-1,-1,1,1,-1,-1,1,-1,-1,1,0,1,-1,-1.0,-1,0,-1,0,1,-1,-1,1,-1,0,-1,-1,0,1,0,1,-1,1,1,-1,0
117,1,1,1,-1,-1,1,-1,1,1,1,-1,1,1,-1,-1,0.0,1,-1,1,-1,1,1,1,1,-1,1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1.0,1,-1,0,1,1,1,1,1,-1,-1,1,-1,-1,1,1,1,-1,-1,-1,1,-1
118,1,-1,1,0,-1,1,1,1,1,0,-1,1,-1,0,-1,-1.0,-1,1,0,1,0,1,-1,1,1,1,1,-1,1,1,0,-1,0,-1,1,-1,-1,1,-1.0,-1,1,0,-1,1,1,-1,-1,0,1,1,-1,-1,1,0,-1,-1,1,0,-1,-1


In [30]:
#val=val.drop(columns=['Question23', 'Question44'])

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [32]:
qq=list(range(1,15))
#qq.remove(23)
#qq.remove(44)
#qq

In [279]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Split the data into training and test sets
ROC=[]
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(train, Y_train1, test_size=0.2)

# Initialize the logistic regression model
    logreg = LogisticRegression(penalty='l1', C=0.16, solver='liblinear')

# Train the logistic regression model
    logreg.fit(X_train, y_train)

# Make predictions on the test set
#
#sub2['I/E']=logreg.predict_proba(val)[:, 1]

# Evaluate the model's performance
    y_pred_proba = logreg.predict_proba(X_test)[:, 1]  # Predict probabilities for the positive class
    yy=logreg.predict_proba(X_train)[:, 1] 
# Calculate AUROC
    auroc = roc_auc_score(y_test, y_pred_proba)
    #print("AUROC:", auroc)
    ROC.append(auroc)
    auroc = roc_auc_score(y_train, yy)
    #print("Train AUROC:", auroc)
    

In [280]:
np.array(ROC).mean()

0.631229075560877

In [363]:
ROC=[]
TROC=[]
for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(train, Y_train3, test_size=0.2)

# Initialize the logistic regression model
    logreg = LogisticRegression(penalty='l1', C=0.7, solver='liblinear')

# Train the logistic regression model
    logreg.fit(X_train, y_train)

# Make predictions on the test set
#
#sub2['I/E']=logreg.predict_proba(val)[:, 1]

# Evaluate the model's performance
    y_pred_proba = logreg.predict_proba(X_test)[:, 1]  # Predict probabilities for the positive class
    yy=logreg.predict_proba(X_train)[:, 1] 
# Calculate AUROC
    auroc = roc_auc_score(y_test, y_pred_proba)
    #print("AUROC:", auroc)
    ROC.append(auroc)
    auroc = roc_auc_score(y_train, yy)
    TROC.append(auroc)

In [364]:
np.array(ROC).mean()

0.4375265451222253

In [365]:
np.array(TROC).mean()

0.9201398835776765

In [344]:
print('Coefficient of each feature:', logreg.coef_)

Coefficient of each feature: [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.06246811 -0.03641897  0.          0.
   0.          0.          0.          0.          0.          0.
   0.09675048  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.22980621  0.
   0.          0.          0.          0.          0.04187239  0.        ]]


In [35]:
stop

NameError: name 'stop' is not defined

In [351]:
train

Q_number,Question1,Question10,Question11,Question12,Question13,Question14,Question15,Question16,Question17,Question18,Question19,Question2,Question20,Question21,Question22,Question23,Question24,Question25,Question26,Question27,Question28,Question29,Question3,Question30,Question31,Question32,Question33,Question34,Question35,Question36,Question37,Question38,Question39,Question4,Question40,Question41,Question42,Question43,Question44,Question45,Question46,Question47,Question48,Question49,Question5,Question50,Question51,Question52,Question53,Question54,Question55,Question56,Question57,Question58,Question59,Question6,Question60,Question7,Question8,Question9
0,-1,-1,1,-1,1,-1,0,-1,0,0,-1,1,-1,0,-1,1.0,1,0,0,-1,1,-1,1,-1,-1,1,-1,-1,1,-1,1,-1,1,1,0,0,-1,-1,0.0,0,0,-1,0,-1,1,0,1,0,-1,1,-1,1,0,0,-1,1,1,0,-1,1
1,0,1,1,1,1,0,1,-1,-1,-1,-1,1,1,1,1,0.0,1,-1,1,0,1,-1,1,-1,-1,0,0,1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1.0,1,-1,1,-1,-1,-1,-1,1,0,-1,-1,-1,1,0,-1,-1,0,-1,0,1,1
2,-1,-1,0,1,1,1,-1,1,0,-1,-1,1,1,0,1,1.0,1,1,-1,1,1,-1,1,-1,0,1,0,1,-1,0,-1,-1,0,0,-1,-1,-1,1,1.0,1,1,1,1,1,1,0,1,1,0,1,-1,1,1,-1,-1,1,1,1,0,-1
3,-1,1,0,-1,0,1,-1,-1,0,1,1,1,-1,1,-1,0.0,0,1,1,0,1,1,1,1,-1,0,1,1,1,-1,1,-1,1,1,-1,-1,1,0,-1.0,-1,-1,-1,-1,1,1,1,1,-1,-1,1,1,0,1,1,-1,1,1,0,1,0
4,1,0,1,1,0,0,0,0,1,1,1,1,-1,-1,-1,1.0,0,0,0,1,1,-1,0,1,1,1,0,0,1,1,1,0,0,-1,1,-1,1,0,1.0,-1,-1,0,1,1,1,1,0,-1,0,1,1,-1,-1,0,-1,0,1,-1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,0,1,-1,0,1,1,-1,1,-1,1,1,-1,-1,-1,-1.0,-1,-1,0,1,0,1,0,1,1,0,1,-1,1,1,1,-1,-1,-1,1,-1,-1,1,0.0,1,-1,0,1,1,1,-1,-1,1,1,1,-1,-1,1,-1,1,0,1,-1,1,-1
116,0,-1,1,0,1,1,1,0,1,0,-1,1,-1,-1,1,1.0,-1,1,-1,-1,1,1,-1,1,0,-1,-1,1,1,-1,-1,1,-1,-1,1,0,1,-1,-1.0,-1,0,-1,0,1,-1,-1,1,-1,0,-1,-1,0,1,0,1,-1,1,1,-1,0
117,1,1,1,-1,-1,1,-1,1,1,1,-1,1,1,-1,-1,0.0,1,-1,1,-1,1,1,1,1,-1,1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1.0,1,-1,0,1,1,1,1,1,-1,-1,1,-1,-1,1,1,1,-1,-1,-1,1,-1
118,1,-1,1,0,-1,1,1,1,1,0,-1,1,-1,0,-1,-1.0,-1,1,0,1,0,1,-1,1,1,1,1,-1,1,1,0,-1,0,-1,1,-1,-1,1,-1.0,-1,1,0,-1,1,1,-1,-1,0,1,1,-1,-1,1,0,-1,-1,1,0,-1,-1


In [352]:
val

Q_number,Question1,Question10,Question11,Question12,Question13,Question14,Question15,Question16,Question17,Question18,Question19,Question2,Question20,Question21,Question22,Question23,Question24,Question25,Question26,Question27,Question28,Question29,Question3,Question30,Question31,Question32,Question33,Question34,Question35,Question36,Question37,Question38,Question39,Question4,Question40,Question41,Question42,Question43,Question44,Question45,Question46,Question47,Question48,Question49,Question5,Question50,Question51,Question52,Question53,Question54,Question55,Question56,Question57,Question58,Question59,Question6,Question60,Question7,Question8,Question9
0,-1,1,1,-1,1,-1,1,1,-1,-1,1,1,-1,0,-1,1,1,-1,1,-1,1,0,-1,-1,1,1,0,-1,1,1,1,0,0,1,-1,-1,0,1,1,1,-1,-1,0,-1,1,1,-1,-1,-1,0,-1,-1,-1,-1,-1,0,-1,-1,-1,1
1,-1,-1,-1,-1,1,1,1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,0,1,0,-1,1,-1,1,-1,-1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,1,1,1,-1,-1,-1,0,1,-1,1,1,1,-1,1,1,-1,0,-1
2,0,-1,0,1,1,-1,-1,1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,0,-1,-1,-1,-1,1,1,1,-1,0,1,-1,-1,-1,1,1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,1,-1,1,1,-1,-1,1,-1,-1,-1,1,1,1,-1,1
3,-1,-1,1,1,1,0,1,0,-1,-1,-1,1,-1,1,-1,-1,1,-1,1,-1,0,-1,1,-1,1,1,0,-1,1,0,1,-1,1,1,1,-1,0,0,0,-1,-1,-1,0,-1,0,-1,-1,-1,1,1,-1,0,-1,1,-1,0,1,-1,-1,1
4,0,-1,1,1,0,0,1,-1,-1,-1,-1,0,-1,-1,1,0,-1,-1,-1,1,1,1,1,1,0,-1,-1,1,1,1,0,-1,1,1,1,-1,0,1,1,1,-1,1,0,-1,1,1,-1,-1,-1,1,0,-1,-1,-1,-1,0,1,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,-1,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,0,1,0,1,1,1,-1,1,-1,0,1,-1,0,1,-1,1,-1,1,-1,-1,1,-1,0,-1,-1,1,1,-1,-1,-1,1,1,1,-1,1,0,0,0,1,-1,1,0,1,1,0
116,0,-1,1,-1,-1,0,1,0,1,-1,-1,1,-1,0,-1,1,-1,0,-1,0,-1,-1,-1,1,0,-1,0,-1,1,-1,1,0,0,1,1,1,1,-1,-1,0,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,1,0,-1,-1,0,1,-1,1,1
117,-1,0,0,1,-1,0,0,1,0,0,-1,-1,1,1,1,0,-1,-1,1,-1,0,0,1,-1,1,1,0,0,0,0,-1,0,-1,0,0,0,-1,0,-1,1,0,1,1,1,-1,0,0,-1,-1,0,0,-1,-1,1,-1,1,0,1,0,0
118,-1,-1,-1,1,1,-1,-1,1,-1,-1,1,1,1,-1,1,1,1,-1,1,0,1,1,0,-1,-1,1,-1,1,-1,-1,-1,1,1,1,1,-1,-1,1,1,1,1,-1,1,-1,1,-1,1,1,-1,1,-1,-1,-1,1,-1,1,1,-1,0,1


In [353]:
subm=pd.read_csv('phase2_test.csv', index_col=0)
subm['User_ID']=subm['User_ID']-240
subm

sub=pd.DataFrame()
sub['User_ID']=subm['User_ID'].unique()
sub['I/E']=0.5
sub['S/N']=0.5
sub['T/F']=0.5
sub['J/P']=0.5

sub

,User_ID,I/E,S/N,T/F,J/P
0,1,0.5,0.5,0.5,0.5
1,2,0.5,0.5,0.5,0.5
2,3,0.5,0.5,0.5,0.5
3,4,0.5,0.5,0.5,0.5
4,5,0.5,0.5,0.5,0.5
...,...,...,...,...,...
115,175,0.5,0.5,0.5,0.5
116,178,0.5,0.5,0.5,0.5
117,179,0.5,0.5,0.5,0.5
118,191,0.5,0.5,0.5,0.5


In [356]:
val

Q_number,Question1,Question10,Question11,Question12,Question13,Question14,Question15,Question16,Question17,Question18,Question19,Question2,Question20,Question21,Question22,Question23,Question24,Question25,Question26,Question27,Question28,Question29,Question3,Question30,Question31,Question32,Question33,Question34,Question35,Question36,Question37,Question38,Question39,Question4,Question40,Question41,Question42,Question43,Question44,Question45,Question46,Question47,Question48,Question49,Question5,Question50,Question51,Question52,Question53,Question54,Question55,Question56,Question57,Question58,Question59,Question6,Question60,Question7,Question8,Question9
0,-1,1,1,-1,1,-1,1,1,-1,-1,1,1,-1,0,-1,1,1,-1,1,-1,1,0,-1,-1,1,1,0,-1,1,1,1,0,0,1,-1,-1,0,1,1,1,-1,-1,0,-1,1,1,-1,-1,-1,0,-1,-1,-1,-1,-1,0,-1,-1,-1,1
1,-1,-1,-1,-1,1,1,1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,0,1,0,-1,1,-1,1,-1,-1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,1,1,1,-1,-1,-1,0,1,-1,1,1,1,-1,1,1,-1,0,-1
2,0,-1,0,1,1,-1,-1,1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,0,-1,-1,-1,-1,1,1,1,-1,0,1,-1,-1,-1,1,1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,1,-1,1,1,-1,-1,1,-1,-1,-1,1,1,1,-1,1
3,-1,-1,1,1,1,0,1,0,-1,-1,-1,1,-1,1,-1,-1,1,-1,1,-1,0,-1,1,-1,1,1,0,-1,1,0,1,-1,1,1,1,-1,0,0,0,-1,-1,-1,0,-1,0,-1,-1,-1,1,1,-1,0,-1,1,-1,0,1,-1,-1,1
4,0,-1,1,1,0,0,1,-1,-1,-1,-1,0,-1,-1,1,0,-1,-1,-1,1,1,1,1,1,0,-1,-1,1,1,1,0,-1,1,1,1,-1,0,1,1,1,-1,1,0,-1,1,1,-1,-1,-1,1,0,-1,-1,-1,-1,0,1,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,-1,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,0,1,0,1,1,1,-1,1,-1,0,1,-1,0,1,-1,1,-1,1,-1,-1,1,-1,0,-1,-1,1,1,-1,-1,-1,1,1,1,-1,1,0,0,0,1,-1,1,0,1,1,0
116,0,-1,1,-1,-1,0,1,0,1,-1,-1,1,-1,0,-1,1,-1,0,-1,0,-1,-1,-1,1,0,-1,0,-1,1,-1,1,0,0,1,1,1,1,-1,-1,0,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,1,0,-1,-1,0,1,-1,1,1
117,-1,0,0,1,-1,0,0,1,0,0,-1,-1,1,1,1,0,-1,-1,1,-1,0,0,1,-1,1,1,0,0,0,0,-1,0,-1,0,0,0,-1,0,-1,1,0,1,1,1,-1,0,0,-1,-1,0,0,-1,-1,1,-1,1,0,1,0,0
118,-1,-1,-1,1,1,-1,-1,1,-1,-1,1,1,1,-1,1,1,1,-1,1,0,1,1,0,-1,-1,1,-1,1,-1,-1,-1,1,1,1,1,-1,-1,1,1,1,1,-1,1,-1,1,-1,1,1,-1,1,-1,-1,-1,1,-1,1,1,-1,0,1


In [354]:
logreg = LogisticRegression(penalty='l1', C=0.16, solver='liblinear')

# Train the logistic regression model
logreg.fit(train, Y_train1)

sub['I/E']=logreg.predict_proba(val)[:, 1]

sub

,User_ID,I/E,S/N,T/F,J/P
0,1,0.371053,0.5,0.5,0.5
1,2,0.520961,0.5,0.5,0.5
2,3,0.375308,0.5,0.5,0.5
3,4,0.504273,0.5,0.5,0.5
4,5,0.400392,0.5,0.5,0.5
...,...,...,...,...,...
115,175,0.647798,0.5,0.5,0.5
116,178,0.510788,0.5,0.5,0.5
117,179,0.421820,0.5,0.5,0.5
118,191,0.326477,0.5,0.5,0.5


In [355]:
sub.to_csv('classicIE.csv', index=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, Y_train2, test_size=0.2, random_state=42)

# Initialize the logistic regression model
logreg = LogisticRegression()

# Train the logistic regression model
logreg.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = logreg.predict_proba(X_test)[:, 1]  # Predict probabilities for the positive class

# Calculate AUROC
auroc = roc_auc_score(y_test, y_pred_proba)
print("AUROC:", auroc)

sub2['S/N']=logreg.predict_proba(val)[:, 1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, Y_train3, test_size=0.2, random_state=42)

# Initialize the logistic regression model
logreg = LogisticRegression()

# Train the logistic regression model
logreg.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = logreg.predict_proba(X_test)[:, 1]  # Predict probabilities for the positive class

# Calculate AUROC
auroc = roc_auc_score(y_test, y_pred_proba)
print("AUROC:", auroc)


sub2['T/F']=logreg.predict_proba(val)[:, 1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, Y_train4, test_size=0.2, random_state=42)

# Initialize the logistic regression model
logreg = LogisticRegression()

# Train the logistic regression model
logreg.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = logreg.predict_proba(X_test)[:, 1]  # Predict probabilities for the positive class

# Calculate AUROC
auroc = roc_auc_score(y_test, y_pred_proba)
print("AUROC:", auroc)


sub2['J/P']=logreg.predict_proba(val)[:, 1]

In [ ]:
sub2

In [ ]:
sub2.to_csv('classic.csv', index=False)